In [1]:
import qutip as qt
from qutip import *
from qutip.qip.operations import hadamard_transform as H, cnot
from qutip.core.operators import identity as I, sigmax as X, sigmay as Y, sigmaz as Z
from qutip.core.states import bell_state

psi0 = rand_ket(2)       # Create a random unknown state (qubit)
psiA = basis(2, 0)       # Alice's initial qubit state |0>
psiB = basis(2, 0)       # Bob's initial qubit state |0>
psi_AB = tensor(H() * psiA, psiB)
psi_AB = cnot() * psi_AB   # Alice and bobs entangled : (|00> + |11>)/√2

#Bell basis states :
phi_plus  = bell_state("00")
phi_minus = bell_state("01")
psi_plus  = bell_state("10")
psi_minus = bell_state("11")
bell_basis = [phi_plus, phi_minus, psi_plus, psi_minus]

def tp_resources():
    return psi0, psi_AB, bell_basis

def teleportation_measurement(psi0, psi_AB, bell_basis, corrs, labels):
    psi_0AB = tensor(psi0, psi_AB)
    res = []
    for i, lab in enumerate(labels):
        proj = bell_basis[i] * bell_basis[i].dag()
        measure_op = tensor(proj, I(2))
        psi_projected = measure_op * psi_0AB
        prob = psi_projected.norm()**2
        if prob > 0:
            psi_normalized = psi_projected.unit()
            dm = psi_normalized.ptrace([2])
            U = corrs[lab]
            rho_bob_corr = U * dm * U.dag()
            fid = qt.metrics.fidelity(rho_bob_corr, psi0 * psi0.dag())
            res.append((lab, prob, fid))
    return res
def main():
    psi0, psi_AB, bell_basis = tp_resources()
    corrs = {
        'phi_plus': I(2),
        'phi_minus': Z(),
        'psi_plus': X(),
        'psi_minus': Z() * X()
    }
    labels = ['phi_plus', 'phi_minus', 'psi_plus', 'psi_minus']
    results = teleportation_measurement(psi0, psi_AB, bell_basis, corrs, labels)
    for lab, prob, fid in results:
        print(f"Outcome: {lab}\n  Prob: {prob:.4f}\n  Fidelity: {fid:.4f}\n")

if __name__ == '__main__':
    main()


Outcome: phi_plus
  Prob: 0.2500
  Fidelity: 1.0000

Outcome: phi_minus
  Prob: 0.2500
  Fidelity: 1.0000

Outcome: psi_plus
  Prob: 0.2500
  Fidelity: 1.0000

Outcome: psi_minus
  Prob: 0.2500
  Fidelity: 1.0000



In [65]:
from qutip import concurrence,basis
psiC = basis(2,0)
psiD = basis(2,0)
psi_CD = tensor(H() * psiC, psiD)
psi_CD = cnot(N=2, control=0, target=1) * psi_CD  # |Φ⁺> for pair CD
psi_ABCD = tensor(psi_AB,psi_CD)
bell_basis = [phi_plus, phi_minus, psi_plus, psi_minus]
def es_resources():
    return psi_AB, psi_CD, bell_basis
def swapping_measurement (psi_ABCD, bell_basis, corrs, labels):
    res = []
    # measure qubits B and C.
    for i, lab in enumerate(labels):
        proj_BC = bell_basis[i] * bell_basis[i].dag()                                # projrct qubit B and C in bell basis states
        measure_op = tensor(qeye(2), proj_BC, qeye(2))                               # Identity on qubit A and D, projector on qubits B & C
        psi_projected = measure_op * psi_ABCD
        prob = psi_projected.norm()**2 
        if prob > 0:
            psi_norm = psi_projected.unit()
            rho_AD = psi_norm.ptrace([0, 3])                                          # Extract the state of qubits A and D 
            # apply the correction on qubit A.
            U = corrs[lab]
            # U acts on A only
            U_ext = tensor(U, qeye(2))
            rho_AD_corr = U_ext * rho_AD * U_ext.dag()
            con=concurrence(rho_AD_corr)
            # Compare with the reference Bell state phi_plus.
            phi_plus = bell_state("00")
            fid = qt.metrics.fidelity(rho_AD_corr, phi_plus * phi_plus.dag())
            res.append((lab, prob, con, fid))
    return res
def main():
    # corrections convert all possible outcomes to phi_plus
    corrs = {
    'phi_plus': I(2),
    'phi_minus': Z(),
    'psi_plus': X(),
    'psi_minus': Z() * X()
    }
    labels = ['phi_plus', 'phi_minus', 'psi_plus', 'psi_minus']
    results = swapping_measurement(psi_ABCD, bell_basis, corrs, labels)
    for lab, prob, fid, con in results:
        print(f"Outcome: {lab}\n  Probability: {prob:.4f}\n  Fidelity with phi_plus: {fid:.4f}\n concurrence with phi_plus : {con:4f}\n")
        

if __name__ == '__main__':
    main()

Outcome: phi_plus
  Probability: 0.2500
  Fidelity with phi_plus: 1.0000
 concurrence with phi_plus : 1.000000

Outcome: phi_minus
  Probability: 0.2500
  Fidelity with phi_plus: 1.0000
 concurrence with phi_plus : 1.000000

Outcome: psi_plus
  Probability: 0.2500
  Fidelity with phi_plus: 1.0000
 concurrence with phi_plus : 1.000000

Outcome: psi_minus
  Probability: 0.2500
  Fidelity with phi_plus: 1.0000
 concurrence with phi_plus : 1.000000



In [3]:
import numpy as np
import qutip as qt
from qutip import *
from qutip.qip.operations import hadamard_transform as H, cnot
from qutip.core.operators import identity as I, sigmax as X, sigmay as Y, sigmaz as Z
from qutip.core.states import bell_state

# Define basis state functions:
phi_l_plus = lambda l: (tensor(basis(2, 0), basis(2, 0)) + l * tensor(basis(2, 1), basis(2, 1))) / np.sqrt(1 + np.abs(l)**2)
phi_l_minus = lambda l: (l.conjugate() * tensor(basis(2, 0), basis(2, 0)) - tensor(basis(2, 1), basis(2, 1))) / np.sqrt(1 + np.abs(l)**2)
psi_p_plus = lambda p: (tensor(basis(2, 0), basis(2, 1)) + p * tensor(basis(2, 1), basis(2, 0))) / np.sqrt(1 + np.abs(p)**2)
psi_p_minus = lambda p: (p.conjugate() * tensor(basis(2, 0), basis(2, 1)) - tensor(basis(2, 1), basis(2, 0))) / np.sqrt(1 + np.abs(p)**2)


def ptp_resources(n):
    psi0 = rand_ket(2)  #unknown state
    phi_AB = tensor(basis(2, 0), basis(2, 0)) + n * tensor(basis(2, 1), basis(2, 1)) #entangled state between Alice and Bob
    phi_AB = phi_AB / phi_AB.norm()
    l = n
    p = n.conjugate()
    basis_states = [phi_l_plus(l), phi_l_minus(l), psi_p_plus(p), psi_p_minus(p)] #set of basis set to make the 
    return psi0, phi_AB, basis_states

def ptp_measurement(psi0, phi_AB, basis, corrs, labels):
    psi_0AB = tensor(psi0, phi_AB)
    target = psi0 * psi0.dag()
    res = []
    for i, lab in enumerate(labels):
        proj = basis[i] * basis[i].dag()
        measure_op = tensor(proj, I(2))
        psi_projected = measure_op * psi_0AB
        prob = expect(measure_op, psi_0AB)
        if prob > 1e-10:
            psi_normalized = psi_projected / np.sqrt(prob)
            dm = psi_normalized.ptrace([2])  # Bob’s uncorrected state
            U = corrs[lab]
            rho_bob_corr = U * dm * U.dag()  # Bob’s corrected state
            fid = qt.metrics.fidelity(rho_bob_corr, target)
            # Store uncorrected state for debugging
            res.append((lab, prob, fid, dm))
    return res

def main():
    # Use a fixed n to ensure non-maximal entanglement
    n = np.random.rand() + 1j * np.random.rand() 
    psi0, phi_AB, basis = ptp_resources(n)
    
    corrs = {
        'phi_plus': I(2),    # No correction
        'phi_minus': Z(),    # σ_z
        'psi_plus': X(),     # σ_x
        'psi_minus': I(2)    # No correction
    }
    labels = ['phi_plus', 'phi_minus', 'psi_plus', 'psi_minus']
    
    results = ptp_measurement(psi0, phi_AB, basis, corrs, labels)
    
    print(f"Resource state parameter n = {n:.4f}")
    print(f"Original state: \n{psi0}")
    print("\nTeleportation Outcomes:")
    for lab, prob, fid, dm in results:
        print(f"Outcome: {lab}")
        print(f"  Probability: {prob:.4f}")
        print(f"  Fidelity: {fid:.4f}")
        print(f"  Bob’s uncorrected state: \n{dm}\n")
    
    n_abs_sq = np.abs(n)**2
    expected_p_succ = 2 * n_abs_sq / (1 + n_abs_sq)**2
    success_prob = sum(prob for lab, prob, _, _ in results if lab in ['phi_minus', 'psi_plus'])
    print(f"Expected success probability (paper): {expected_p_succ:.4f}")
    print(f"Computed success probability (phi_minus + psi_plus): {success_prob:.4f}")

if __name__ == "__main__":
    main()

Resource state parameter n = 0.6478+0.1376j
Original state: 
Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[-0.31805427+0.54902906j]
 [ 0.75630019-0.15943211j]]

Teleportation Outcomes:
Outcome: phi_plus
  Probability: 0.2501
  Fidelity: 0.9239
  Bob’s uncorrected state: 
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.77792446+0.j        -0.27805286+0.3089411j]
 [-0.27805286-0.3089411j  0.22207554+0.j       ]]

Outcome: phi_minus
  Probability: 0.2119
  Fidelity: 1.0000
  Bob’s uncorrected state: 
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.40259142+0.j         0.32807737-0.36452272j]
 [0.32807737+0.36452272j 0.59740858+0.j        ]]

Outcome: psi_plus
  Probability: 0.2119
  Fidelity: 1.0000
  Bob’s uncorrected state: 
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.59740858+0.j      